In [40]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time
import os
import pandas as pd

companies = pd.read_excel("companies.xlsx")

chrome_options = webdriver.ChromeOptions()
settings = {
       "recentDestinations": [{
            "id": "Save as PDF",
            "origin": "local",
            "account": "",
        }],
        "selectedDestinationId": "Save as PDF",
        "version": 2
    }
prefs = {'printing.print_preview_sticky_settings.appState': json.dumps(settings)}
chrome_options.add_experimental_option('prefs', prefs)
chrome_options.add_argument('--kiosk-printing')
chrome_options.add_argument('--start-maximized')

driver = webdriver.Chrome('./chromedriver/chromedriver.exe', chrome_options=chrome_options)

f5_url = 'https://fortune.com/fortune500/2020/search/'

driver.get(f5_url)

rows_xpath = '//*[@aria-label="rows per page"]/option[text()="100 rows"]'
driver.find_element_by_xpath(rows_xpath).click()

# remove_xpath = '//*[text()="Remove"]'
# element = WebDriverWait(driver, 10).until(
#     EC.element_to_be_clickable((By.XPATH, remove_xpath))
# )

# for i in range(9):
#     driver.find_element_by_xpath(remove_xpath).click()

In [53]:
# To download names of company from Forbes
companies = []
for i in range(10):
    results_xpath = '//*[@class="searchResults__cellContent--3WEWj"]'
    elements = driver.find_elements_by_xpath(results_xpath)

    for i in range(1100):
        if i%11 == 1:
            companies.append(elements[i].text)

    next_xpath = '//*[text()="Next"]'
    
    wait = WebDriverWait(driver, 10)
    element = wait.until(EC.element_to_be_clickable((By.XPATH, next_xpath)))
    
    driver.find_element_by_xpath(next_xpath).click()

TimeoutException: Message: 


In [36]:
import pandas as pd
df = pd.DataFrame(companies, columns=['Company Name'])
df.to_excel("companies.xlsx", index=False)

In [37]:
cmp = list(companies["Company Name"][0:2])
failed = []
sec_url = 'https://www.sec.gov/edgar/searchedgar/companysearch.html'

searchbar_xpath = '//*[@id="company"]'
searchbtn_xpath = '//*[@id="search_button"]'

for company in cmp:
    try:
        driver.get(sec_url)
        driver.find_element_by_xpath(searchbar_xpath).send_keys(company)
        driver.find_element_by_xpath(searchbtn_xpath).click()

        cik_xpath = '(//tr[2])/td[@valign="top"]'
        wait = WebDriverWait(driver, 10)
        cik = wait.until(EC.element_to_be_clickable((By.XPATH, cik_xpath)))
        cik = ((driver.find_element_by_xpath(cik_xpath)).text)
        cik1 = cik.split()[0]
        cik2 = cik.split()[-1]

        if cik1.isnumeric():
            cik = cik1
        elif cik2.isnumeric():
            cik = cik2

        cik_url = 'https://www.sec.gov/edgar/browse/?CIK='+cik+'&owner=exclude'
        driver.get(cik_url)

        dropdown_xpath = '(//h5[@class="card-header bg-sec-silver collapsible"])[2]'
        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.element_to_be_clickable((By.XPATH, dropdown_xpath)))
        driver.find_element_by_xpath(dropdown_xpath).click()

        file_xpath = '(//div[@id="selected-filings-annualOrQuarterly"]//li)[1]'
        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.element_to_be_clickable((By.XPATH, file_xpath)))
        driver.find_element_by_xpath(file_xpath).click()

        driver.switch_to.window(driver.window_handles[-1])
        # driver.execute_script('window.print();')
        new_url = driver.current_url
        new_url = new_url.replace('ix?doc=/', '')
        driver.get(new_url)

        time.sleep(5)
        driver.execute_script('window.print();')

        time.sleep(5)
        src = 'C:/Users/apoor/Downloads/'+(new_url.split('/')[-1]).replace('.htm', '.pdf')
        dest = './files/'+company+'.pdf'
        os.rename(src, dest)

        driver.close()
        driver.switch_to.window(driver.window_handles[-1])
    except:
        failed.append(company)
        continue

In [14]:
failed

['Apple',
 'CVS Health',
 'Berkshire Hathaway',
 'UnitedHealth Group',
 'McKesson',
 'AT&T',
 'AmerisourceBergen']

In [45]:
cmp = list(companies["Company Name"][10:250])
failed = []
sec_url = 'https://www.sec.gov/edgar/searchedgar/companysearch.html'

searchbar_xpath = '//*[@id="company"]'
searchbtn_xpath = '//*[@id="search_button"]'

for company in cmp:
    try:
        driver.get(sec_url)
        driver.find_element_by_xpath(searchbar_xpath).send_keys(company)
        time.sleep(5)
        el = driver.find_element_by_xpath(searchbar_xpath)
        action = webdriver.common.action_chains.ActionChains(driver)
        action.move_to_element_with_offset(el, 100, 90)
        action.click()
        action.perform()
        dropdown_xpath = '(//*[contains(text(),"10-K")])[1]'
        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.element_to_be_clickable((By.XPATH, dropdown_xpath)))
        driver.find_element_by_xpath(dropdown_xpath).click()

        file_xpath = '(//*[contains(text(),"10-K")])[2]'
        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.element_to_be_clickable((By.XPATH, file_xpath)))
        driver.find_element_by_xpath(file_xpath).click()
        
        driver.switch_to.window(driver.window_handles[-1])
        # driver.execute_script('window.print();')
        new_url = driver.current_url
        new_url = new_url.replace('ix?doc=/', '')
        driver.get(new_url)

        time.sleep(5)
        driver.execute_script('window.print();')

        time.sleep(5)
        src = 'C:/Users/apoor/Downloads/'+(new_url.split('/')[-1]).replace('.htm', '.pdf')
        dest = './files/'+company+'.pdf'
        os.rename(src, dest)

        driver.close()
        driver.switch_to.window(driver.window_handles[-1])
    except:
        print(company)
        failed.append(company)
        if len(driver.window_handles) == 1:
            continue
        else:            
            driver.close()
            driver.switch_to.window(driver.window_handles[-1])
        continue

Cardinal Health
JPMorgan Chase
Microsoft
Fannie Mae
Bank of America
Comcast NBCUniversal
Wells Fargo
State Farm Insurance
Freddie Mac
Lowe's
FedEx
MetLife
Walt Disney
Prudential Financial (U.S.)
Albertsons
HP
Goldman Sachs Group
Morgan Stanley
Cisco
HCA Healthcare
American International Group
New York Life Insurance
Liberty Mutual Insurance Group
Tyson Foods
TIAA
Oracle
Deere
Nike
Massachusetts Mutual Life
Tech Data
USAA
Exelon
Northwestern Mutual
Travelers Cos.
U.S. Bancorp
Paccar
Macy's
Jabil
Tesla
NGL Energy Partners
Synnex
United Natural Foods
Micron Technology
Nucor
PNC Financial Services Group
Southern Company
DXC Technology
McDonald's
ManpowerGroup
Bank of New York Mellon
Kohl's
Performance Food Group
CarMax
WestRock
D.R. Horton
Lincoln National
General Mills
Dominion Energy
Principal Financial
American Electric Power
C.H. Robinson Worldwide
Estée Lauder
Goodyear Tire & Rubber
BlackRock
Parker-Hannifin
Automatic Data Processing
Land O'Lakes
VF
Guardian Life Ins. Co. of America
C

In [60]:
driver.quit()

In [26]:
companies["Company Name"][0:10]

0               Walmart
1                Amazon
2           Exxon Mobil
3                 Apple
4            CVS Health
5    Berkshire Hathaway
6    UnitedHealth Group
7              McKesson
8                  AT&T
9     AmerisourceBergen
Name: Company Name, dtype: object

In [58]:
import glob
files = glob.glob('./files/*.pdf')
files[0:10]

['./files\\3M.pdf',
 './files\\Abbott Laboratories.pdf',
 './files\\AbbVie.pdf',
 './files\\AECOM.pdf',
 './files\\Aflac.pdf',
 './files\\Albertsons.pdf',
 './files\\Allstate.pdf',
 './files\\Alphabet.pdf',
 './files\\Altria Group.pdf',
 './files\\Amazon.pdf']

In [59]:
count=0
for failed_f in failed:
    temp = './files\\'+failed_f+'.pdf'
    if temp not in files:
        print(temp)
        count+=1
count

./files\JPMorgan Chase.pdf
./files\Fannie Mae.pdf
./files\Bank of America.pdf
./files\Comcast NBCUniversal.pdf
./files\Wells Fargo.pdf
./files\State Farm Insurance.pdf
./files\Freddie Mac.pdf
./files\Lowe's.pdf
./files\FedEx.pdf
./files\MetLife.pdf
./files\Walt Disney.pdf
./files\Prudential Financial (U.S.).pdf
./files\HP.pdf
./files\Goldman Sachs Group.pdf
./files\Morgan Stanley.pdf
./files\American International Group.pdf
./files\New York Life Insurance.pdf
./files\Liberty Mutual Insurance Group.pdf
./files\Tyson Foods.pdf
./files\TIAA.pdf
./files\Oracle.pdf
./files\Deere.pdf
./files\Massachusetts Mutual Life.pdf
./files\USAA.pdf
./files\Exelon.pdf
./files\Northwestern Mutual.pdf
./files\Travelers Cos..pdf
./files\U.S. Bancorp.pdf
./files\Macy's.pdf
./files\Jabil.pdf
./files\United Natural Foods.pdf
./files\Micron Technology.pdf
./files\PNC Financial Services Group.pdf
./files\Southern Company.pdf
./files\McDonald's.pdf
./files\Bank of New York Mellon.pdf
./files\Kohl's.pdf
./files\D

50